In [3]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

In [5]:
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Mnist first try

    1、单层MLP，输出用自己softmax进行放缩。
    2、loss选择cross-entropy

In [22]:
'''
图像用784维的向量表示
label用10维的向量表示
我们需要将一个784维的向量映射到10维的向量上

一层神经网络

'''
# 加载数据
image_size=784
label_size=10
# 所以这里下次还不是要命名为这个train后缀了，太有迷惑性了
# 命名为input可能会好一点，然后在train和test的步骤分别输入不同的数据即可 
x_input=tf.placeholder(shape=(None,image_size),dtype=tf.float32)
y_input=tf.placeholder(shape=(None,label_size),dtype=tf.float32)

# 简单神经网络
# 相当于使用一个给定size的矩阵来初始化W
W=tf.Variable(initial_value=tf.zeros([image_size,label_size]))
b=tf.Variable(initial_value=tf.zeros([label_size]))

y_output=tf.nn.softmax(tf.matmul(x_input,W)+b)

# 还有loss定义

loss=-tf.reduce_sum(y_input*tf.log(y_output))
train_step=tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

In [24]:
# 评估
# 第一项是预测值，第二项是真值。根据输入的内容不同来计算
correct_place=tf.equal(tf.argmax(y_output,1),tf.argmax(y_input,1))

accuracy=tf.reduce_mean(tf.cast(correct_place,'float'))


In [25]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        batch_x,batch_y=mnist.train.next_batch(100)
        result=sess.run(train_step,feed_dict={x_input:batch_x,y_input:batch_y})
        if i% 50 ==0:
            print(i,sess.run(loss,feed_dict={x_input:batch_x,y_input:batch_y}))
    
    temp_accu=sess.run(accuracy,feed_dict={x_input:mnist.test.images,
                                           y_input:mnist.test.labels})
    print(temp_accu) # 我去，loss这么高还有这个尿性= - 

0 148.45375
50 29.131947
100 23.077423
150 24.698631
200 32.699097
250 25.220123
300 18.57993
350 21.53984
400 20.046234
450 15.719261
500 17.975788
550 11.018141
600 17.187658
650 16.328846
700 23.269932
750 23.074778
800 20.17444
850 15.583145
900 17.320763
950 14.836184
0.9162


## Mnist second try：

    1、Convolution

In [48]:
# 初始化参数
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    '''
    bias的初始化会有比较大的影响，如果采用truncated_nomal初始化的话
    模型貌似是不能收敛的
    '''
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [49]:
# 定义卷积层和池化层
def conv2d(x,W):
    '''
     x表示输入图像，具体定义：[batch, in_height, in_width, in_channels]`
     W表示自定义的卷积核，具体定义如下：[filter_height, filter_width, in_channels, out_channels]
     strids表示步长的维度信息，定义如下：[filter_height, filter_width, in_channels, out_channels]
    '''
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    '''
    x：[batch, in_height, in_width, in_channels]4个维度的
    ksize表示pool所有核的维度信息，4个参数对应于输入x的四个维度的pool
    stride：表示步长的维度信息
    '''
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')



In [50]:
'''
input_layer
'''
x_input=tf.placeholder(shape=(None,784),dtype=tf.float32)
y_input=tf.placeholder(shape=(None,10),dtype=tf.float32)

In [51]:
'''
first layer conv
这里比较奇怪，相当于我们要先将卷积核定义好，然后再输入
''' 
# 第一层卷积核size为5*5，输入图像channel为1，输出feature-map的channel为32
W_conv1=weight_variable([5,5,1,32])
b_conv1=bias_variable([32])

# 把输入变回28*28的图像
x_image=tf.reshape(x_input,shape=[-1,28,28,1])

# 做卷积
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [52]:
'''
second layer conv
'''
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [53]:
'''
Dense layer
'''
W_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])

h_pool2_flat=tf.reshape(h_pool2,shape=[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

In [54]:
'''
Drop layer
'''
keep_prob=tf.placeholder(dtype=tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

In [55]:
'''
output_layer
'''
W_fc2=weight_variable([1024,10])
b_fc2=weight_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

In [56]:
'''
define loss
cross entropy
'''
loss=-tf.reduce_sum(y_input*tf.log(y_conv))
trainer=tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)

In [63]:
'''
evaluate model
'''
# 所以之前准确率低是这里的写错了
correct_pred=tf.equal(tf.argmax(y_conv,1),tf.argmax(y_input,1))
# 计算均值，cast之后的correct_pred相当于是【0,0,0,1,1,1,0】这样的数组，
# 算出它的均值就是对应的accuracy了
accuracy=tf.reduce_mean(tf.cast(correct_pred,'float'))

In [64]:
'''
train
'''
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(20000):
        batch_x,batch_y=mnist.train.next_batch(50)
        sess.run(trainer,feed_dict={x_input:batch_x,
                                    y_input:batch_y,
                                    keep_prob:0.5})
        if i%100==0:
            result=sess.run(accuracy,feed_dict={x_input:batch_x,
                                                y_input:batch_y,
                                                keep_prob:1})
            print(int(i/100),result)
    batch_test_x=mnist.test.images
    batch_test_y=mnist.test.labels
    all_test=sess.run(accuracy,feed_dict={x_input:batch_test_x,
                                          y_input:batch_test_y,
                                          keep_prob:1.0})
    print(all_test)

0 0.12
1 0.82
2 0.86
3 0.94
4 0.94
5 0.96
6 1.0
7 0.9
8 0.98
9 0.98
10 1.0
11 0.92
12 0.96
13 0.96
14 0.98
15 1.0
16 0.96
17 0.98
18 1.0
19 0.96
20 0.96
21 0.96
22 1.0
23 0.96
24 1.0
25 0.96
26 1.0
27 1.0
28 0.98
29 1.0
30 0.94
31 1.0
32 0.98
33 0.96
34 0.94
35 1.0
36 1.0
37 0.98
38 1.0
39 1.0
40 1.0
41 0.98
42 0.98
43 1.0
44 0.98
45 1.0
46 0.98
47 1.0
48 1.0
49 0.98
50 1.0
51 1.0
52 1.0
53 1.0
54 0.98
55 0.98
56 1.0
57 0.98
58 1.0
59 1.0
60 1.0
61 1.0
62 0.98
63 0.98
64 0.98
65 0.98
66 0.98
67 1.0
68 1.0
69 0.96
70 1.0
71 1.0
72 1.0
73 1.0
74 1.0
75 1.0
76 1.0
77 0.98
78 0.98
79 1.0
80 1.0
81 1.0
82 0.98
83 1.0
84 1.0
85 1.0
86 1.0
87 0.98
88 1.0
89 0.98
90 1.0
91 0.98
92 1.0
93 1.0
94 0.98
95 1.0
96 1.0
97 1.0
98 0.98


KeyboardInterrupt: 